### Adjust Word Group In Youtube Deploy Result

In [1]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [2]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [3]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "Arabic"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# source path
source_path = "/home/kurubal/Downloads/Temp"

# parameter
file = "69 Sentence Selected"  # file without extention
sheets = "Sheet1"  # 2 gram target, 2 gram hybrid
file_ext = "69_Sentence_Review"
first_column = 0  # start column location number
end_column = 1

# native word select for Part 2 
word_start = 0  # 0  # native word start index
word_end = 200 # 28  # native word end index

# word all usage in sent
word_use_big_than = 2

In [4]:
def word_count_result(df,column_list): # df is dataframe, column_list is list value
    '''
    word_count_bool(df, column_list): df columns word count for word frequency\n
    df is dataframe, column_list is list value\n
    word_count_bool(df, ["word","twogram"]):
    '''
    list_all = []
    for i in df.loc[:,[x for x in column_list]].columns:
        var_list = df[f"{i}"].dropna().tolist()
        for j in var_list:
            list_all.append(j)
    text = " ".join(list_all)
    word_list = re.findall(r"\w+",text, re.UNICODE)
    df_word_list = pd.DataFrame(word_list, columns=["word"])
    #df_word_list.rename(columns={0:"word"}, inplace=True)
    df_word_count = pd.DataFrame(df_word_list.value_counts())
    df_word_count.reset_index(inplace=True)
    df_word_count.rename(columns={0:"word_count"}, inplace=True)
    
    return  df_word_count

In [5]:
def word_in_wordgroup(source_word_list, df_target, target_column, word_sample_num, simple=False):

    '''word_in_wordgroup(not_in_sent_word_list, df_youtube_sent_select, "search_string", 5, simple=False)\n
       source_word_list is searching word list\n
       df_target is dataframe, target_column are dataframe column string name\n
       word_sample_num is searching sample number.
       simple use for all column result or only target column result 
    '''
    if simple:
        df_select = df_target[[f"{target_column}"]].dropna()
    else:
        df_select = df_target
        
    df_result = pd.DataFrame()
    for i in source_word_list:
        try:
            word_in_word_cluster = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)].sample(word_sample_num)    
        except:
            word_in_word_cluster = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)].head(word_sample_num)
            #pass        
        word_in_word_cluster.insert(0,"word",i)
        df_result = pd.concat([df_result,word_in_word_cluster], axis=0)
    df_result.reset_index(drop=True, inplace=True)

    return df_result

In [6]:
#path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
#Deployment2/Result/8-Adjust Word Group In Youtube Deploy Result/{lang_folder.capitalize()}"

path = "/home/kurubal/Downloads/Temp"

Path(path).mkdir(parents=True, exist_ok=True)

In [7]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
987922,karneleme,5
987923,karnaya,5
987924,dörtlümüzün,5
987925,karnavalınız,5


In [8]:
df_word_select = df_word_all.iloc[word_start:word_end,]
df_word_select.reset_index(inplace=True)
df_word_select

,index,word,frequency
0,0,bir,18835735
1,1,bu,11062659
2,2,ne,8025880
3,3,ve,7766036
4,4,için,5484109
...,...,...,...
195,195,saat,399989
196,196,onunla,399330
197,197,yapıyorsun,398274
198,198,neler,397377


In [9]:
df_file = pd.read_excel(f"{source_path}/{file}.xlsx", sheet_name=f"{sheets}")
#df_file.sort_values("search_string",key=lambda x:x.str.len(), inplace=True, ascending=True)
df_file.reset_index(drop=True, inplace=True)
df_file

,search_string
0,kendi şimdi sen mi ne oluyor biliyor musun bu ...
1,aynı şeyi tekrar tekrar
2,ne kadar büyük bir para değil mi
3,gerçekten uzun ve bu da benim için çok önemli ...
4,en son şu dakika önce
5,zaten birlikte olmaz zaten geri
6,her zaman öyle değil herkes için de her zaman ...
7,var mı bir yardım var mı diye böyle
8,oh ne güzel çok güzel ama sonra
9,bile yok bu da benim için aslında önemli bir ş...


In [10]:
df_file_count = word_count_result(df_file, df_file.columns[0:1])  # "search_string" column
df_check = df_file_count[df_file_count["word_count"] > word_use_big_than]
check_list = df_check["word"].to_list()
remove_list = []
df_file_var = df_file
for var_sent in df_file["search_string"]:
    result_list = []
    words = re.findall(r"\w+",var_sent, re.UNICODE)
    for word in words:
        if word in check_list:
            result_list.append(True)
        else:
            result_list.append(False)
    if all(result_list):
        df_file_var = df_file_var[df_file_var["search_string"] != f"{var_sent}"]
        df_file_var_count = word_count_result(df_file_var, df_file_var.columns[0:1])
        df_check = df_file_var_count[df_file_var_count["word_count"] > word_use_big_than]
        check_list = df_check["word"].to_list()
        remove_list.append(var_sent)
    else:
        pass

df_file_var.reset_index(drop=True, inplace=True)
df_file_var

,search_string
0,kendi şimdi sen mi ne oluyor biliyor musun bu ...
1,aynı şeyi tekrar tekrar
2,ne kadar büyük bir para değil mi
3,gerçekten uzun ve bu da benim için çok önemli ...
4,en son şu dakika önce
5,zaten birlikte olmaz zaten geri
6,her zaman öyle değil herkes için de her zaman ...
7,var mı bir yardım var mı diye böyle
8,oh ne güzel çok güzel ama sonra
9,bile yok bu da benim için aslında önemli bir ş...


In [11]:
remove_list

[]

In [12]:
df_result_word_count = word_count_result(df_file_var, df_file.columns[0:1])
df_result_word_count

,word,word_count
0,bir,15
1,çok,11
2,için,7
3,değil,7
4,ne,7
...,...,...
177,hayır,1
178,harika,1
179,hakkında,1
180,hadi,1


In [13]:
df_result_word_count["word_count"].sum()

381

In [14]:
#df_file_var.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Group_In_Youtube_Sentence_Word_Index_List_Sample_Selected.xlsx", index=False)

##### Not In Sent And Result Word Count

In [15]:
set_result_word_count = set(df_result_word_count["word"])
set_word_select = set(df_word_select["word"])

In [16]:
not_in_sent_word_list = list(set_word_select.difference(set_result_word_count))
not_in_sent_word_list

['iş',
 'onunla',
 'bize',
 'haydi',
 'lazım',
 'kız',
 'sana',
 'olacak',
 'değilim',
 'pekala',
 'şeyler',
 'iyi',
 'olmak',
 'bak',
 'olan',
 'ile',
 'biliyorsun',
 'hala']

In [17]:
df_youtube_sent_result = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment2/Result/\
2-Word Group In Youtube Sentence/{lang_folder.lower().capitalize()}/{lang_folder.lower().capitalize()}_Word_Group_Max_In_Youtube_Sentence_200_Word6.csv")
df_youtube_sent_result

,search_string,start_time,end_time,sentence,video_id,video_url
0,bilmiyorum belki hiçbir şey olmaz belki çok şe...,381,385,bilmiyorum belki hiçbir şey olmaz belki çok şe...,L3bf7dlR5hk,https://www.youtube.com/watch?v=L3bf7dlR5hk&t=...
1,yardım yapıyorsun çok teşekkürler çok zaman he...,1222,1230,yardım yapıyorsun çok teşekkürler çok zaman he...,zgQbOmPp9pk,https://www.youtube.com/watch?v=zgQbOmPp9pk&t=...
2,ya böyle bir şey olabilir mi ya ama bir şey ol...,466,480,ya böyle bir şey olabilir mi ya ama bir şey ol...,duaQwKTmgvE,https://www.youtube.com/watch?v=duaQwKTmgvE&t=...
3,çok güzel bir dakika bir dakika bir dakika bir...,525,531,çok güzel bir dakika bir dakika bir dakika bir...,OZhgDD1ScBg,https://www.youtube.com/watch?v=OZhgDD1ScBg&t=...
4,senin için önemli olan tek şey hala burada bil...,84,88,senin için önemli olan tek şey hala burada bil...,9Wx3JJTq_Hw,https://www.youtube.com/watch?v=9Wx3JJTq_Hw&t=84s
...,...,...,...,...,...,...
190803,ol o ne o,3071,3079,abone ol o ne o abone ol,DbsR_Cccadg,https://www.youtube.com/watch?v=DbsR_Cccadg&t=...
190804,ol o o o,2258,2260,abone ol o o o,K0b_9Xzy79A,https://www.youtube.com/watch?v=K0b_9Xzy79A&t=...
190805,mi o o o,419,428,mi o o o,5ofdGcvaeZU,https://www.youtube.com/watch?v=5ofdGcvaeZU&t=...
190806,o ne o o,3947,3948,o ne o o i lyasın yanındaki ayşen mi ya,D6Va3TXwFEQ,https://www.youtube.com/watch?v=D6Va3TXwFEQ&t=...


In [18]:
disable_videoid_list = ["P5bYtFrqw68","paJ2IFbflzU","Wp7JJK2jToU","zG40xV3dAuE","WqRjVy_YAtY","x7uac1mes8c","bUr_cw89CRo","vO7b6xnszGM"]

In [19]:
df_youtube_sent_result = df_youtube_sent_result[~df_youtube_sent_result["video_id"].isin(disable_videoid_list)]
df_youtube_sent_result

,search_string,start_time,end_time,sentence,video_id,video_url
0,bilmiyorum belki hiçbir şey olmaz belki çok şe...,381,385,bilmiyorum belki hiçbir şey olmaz belki çok şe...,L3bf7dlR5hk,https://www.youtube.com/watch?v=L3bf7dlR5hk&t=...
1,yardım yapıyorsun çok teşekkürler çok zaman he...,1222,1230,yardım yapıyorsun çok teşekkürler çok zaman he...,zgQbOmPp9pk,https://www.youtube.com/watch?v=zgQbOmPp9pk&t=...
2,ya böyle bir şey olabilir mi ya ama bir şey ol...,466,480,ya böyle bir şey olabilir mi ya ama bir şey ol...,duaQwKTmgvE,https://www.youtube.com/watch?v=duaQwKTmgvE&t=...
3,çok güzel bir dakika bir dakika bir dakika bir...,525,531,çok güzel bir dakika bir dakika bir dakika bir...,OZhgDD1ScBg,https://www.youtube.com/watch?v=OZhgDD1ScBg&t=...
4,senin için önemli olan tek şey hala burada bil...,84,88,senin için önemli olan tek şey hala burada bil...,9Wx3JJTq_Hw,https://www.youtube.com/watch?v=9Wx3JJTq_Hw&t=84s
...,...,...,...,...,...,...
190803,ol o ne o,3071,3079,abone ol o ne o abone ol,DbsR_Cccadg,https://www.youtube.com/watch?v=DbsR_Cccadg&t=...
190804,ol o o o,2258,2260,abone ol o o o,K0b_9Xzy79A,https://www.youtube.com/watch?v=K0b_9Xzy79A&t=...
190805,mi o o o,419,428,mi o o o,5ofdGcvaeZU,https://www.youtube.com/watch?v=5ofdGcvaeZU&t=...
190806,o ne o o,3947,3948,o ne o o i lyasın yanındaki ayşen mi ya,D6Va3TXwFEQ,https://www.youtube.com/watch?v=D6Va3TXwFEQ&t=...


In [20]:
df_not_in_sent_selected_word_list_result = word_in_wordgroup(not_in_sent_word_list, df_youtube_sent_result, "search_string", 50000, simple=False)
#df_not_in_sent_selected_word_list_result = not_in_sent_selected_word_list_result.drop(["word"], axis=1)
df_not_in_sent_selected_word_list_result

,word,search_string,start_time,end_time,sentence,video_id,video_url
0,iş,iyi de sen böyle bir iş için biraz küçük değil...,4787,4790,tatlım iyi de sen böyle bir iş için biraz küçü...,vleTt0rWGw4,https://www.youtube.com/watch?v=vleTt0rWGw4&t=...
1,iş,senin için daha iyi olur çünkü demek ki daha ç...,1425,1429,senin için daha iyi olur çünkü demek ki daha ç...,1iByM08MvqU,https://www.youtube.com/watch?v=1iByM08MvqU&t=...
2,iş,o zaman dur ya o zaman ben artık bunu iş olarak,1028,1032,o zaman dur ya o zaman ben artık bunu iş olara...,tENkclsn4JY,https://www.youtube.com/watch?v=tENkclsn4JY&t=...
3,iş,iş olabilir başka bir şey olabilir ama burada,317,321,iş olabilir başka bir şey olabilir ama burada ...,BZ80_0ARk6w,https://www.youtube.com/watch?v=BZ80_0ARk6w&t=...
4,iş,tamam ya tamam oldu bu iş işte ne güzel tamam,1897,1901,tamam ya tamam oldu bu iş işte ne güzel tamam,Bbaal2Neohk,https://www.youtube.com/watch?v=Bbaal2Neohk&t=...
...,...,...,...,...,...,...,...
39785,hala,var o da hala,409,411,tek büyük yanlış var o da hala sizin bu evde o...,q3aI7Eds5Qo,https://www.youtube.com/watch?v=q3aI7Eds5Qo&t=...
39786,hala,bu ne hala ya,2251,2253,bu ne hala ya,RT602tXUlEA,https://www.youtube.com/watch?v=RT602tXUlEA&t=...
39787,hala,ki siz hala o,7728,7730,ama görüyorum ki siz hala o günlerde kalmışsınız,U9cyavnOr1E,https://www.youtube.com/watch?v=U9cyavnOr1E&t=...
39788,hala,o hala yok o,3729,3730,o hala yok o şartım var bu şartım var diyecek,H5tCfLoE_UM,https://www.youtube.com/watch?v=H5tCfLoE_UM&t=...


In [21]:
df_test = df_not_in_sent_selected_word_list_result[["word","video_id"]]
df_test.drop_duplicates(inplace=True)
df_test

/home/kurubal/anaconda3/envs/py39/lib/python3.9/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


,word,video_id
0,iş,vleTt0rWGw4
1,iş,1iByM08MvqU
2,iş,tENkclsn4JY
3,iş,BZ80_0ARk6w
4,iş,Bbaal2Neohk
...,...,...
39783,hala,KsZ_8oIQsv4
39784,hala,3CyjDc1nWA4
39785,hala,q3aI7Eds5Qo
39787,hala,U9cyavnOr1E


In [22]:
df_test["video_id"].value_counts(ascending=False)

zgQbOmPp9pk    15
TVy2hkF_G9c    14
Ws5feMfuQ7I    14
2YqbKDoFyL4    14
HhrBZuxjFxM    14
               ..
_Tr90HKsUQ4     1
nmSxYYo2Qss     1
Zb3Ag9_KE1E     1
TZnFotgu_XY     1
3CyjDc1nWA4     1
Name: video_id, Length: 5914, dtype: int64

In [23]:
df_test[df_test["video_id"] == "zgQbOmPp9pk"]

,word,video_id
47,iş,zgQbOmPp9pk
2791,onunla,zgQbOmPp9pk
3564,bize,zgQbOmPp9pk
4932,lazım,zgQbOmPp9pk
6294,kız,zgQbOmPp9pk
8069,sana,zgQbOmPp9pk
14531,olacak,zgQbOmPp9pk
18418,değilim,zgQbOmPp9pk
18867,şeyler,zgQbOmPp9pk
21468,iyi,zgQbOmPp9pk


In [24]:
df_test2 = df_not_in_sent_selected_word_list_result[["search_string","start_time","end_time","sentence","video_id"]]
df_test2

,search_string,start_time,end_time,sentence,video_id
0,iyi de sen böyle bir iş için biraz küçük değil...,4787,4790,tatlım iyi de sen böyle bir iş için biraz küçü...,vleTt0rWGw4
1,senin için daha iyi olur çünkü demek ki daha ç...,1425,1429,senin için daha iyi olur çünkü demek ki daha ç...,1iByM08MvqU
2,o zaman dur ya o zaman ben artık bunu iş olarak,1028,1032,o zaman dur ya o zaman ben artık bunu iş olara...,tENkclsn4JY
3,iş olabilir başka bir şey olabilir ama burada,317,321,iş olabilir başka bir şey olabilir ama burada ...,BZ80_0ARk6w
4,tamam ya tamam oldu bu iş işte ne güzel tamam,1897,1901,tamam ya tamam oldu bu iş işte ne güzel tamam,Bbaal2Neohk
...,...,...,...,...,...
39785,var o da hala,409,411,tek büyük yanlış var o da hala sizin bu evde o...,q3aI7Eds5Qo
39786,bu ne hala ya,2251,2253,bu ne hala ya,RT602tXUlEA
39787,ki siz hala o,7728,7730,ama görüyorum ki siz hala o günlerde kalmışsınız,U9cyavnOr1E
39788,o hala yok o,3729,3730,o hala yok o şartım var bu şartım var diyecek,H5tCfLoE_UM


In [25]:
df_test2[df_test2.duplicated()]

,search_string,start_time,end_time,sentence,video_id
29,da bir iş var ama dur bakalım ne var,3609,3612,o i drisin olayında da bir iş var ama dur baka...,8_VkxmHeMAk
173,göre bir iş ne oluyor burada,7255,7258,i şte tam hirama göre bir iş ne oluyor burada,HMrN12C4JwM
341,bakalım bu iş böyle güzel,1821,1823,bakalım bu iş böyle güzel işler mi kıyak işler...,RPJg_lUfHvI
419,nasıl olacak bu iş böyle,1864,1866,nasıl olacak bu iş böyle i yi ki bir işe girdin,0Rtmbtf7piU
529,yani nasıl olacak bu iş,7004,7007,yani nasıl olacak bu iş hünk rım vallahi zor iş,tsI6CFPkguQ
...,...,...,...,...,...
39724,ya sana hala ne,6373,6375,sebebini söyledim ya sana hala ne diyorsun,orEhowBkuKI
39728,bu kız hala çok,7690,7692,laf aramızda bu kız hala çok konuşuyor hazar bey,2YqbKDoFyL4
39743,ya hala bak bir,4375,4377,ya hala bak bir bildiğin varsa,6GC8ekmdngk
39744,ya hala iyi kız,1761,1762,i nanmıyorum ya hala iyi kız rolü yapıyorlar,pRbi4o3O038


In [26]:
#df_not_in_sent_selected_word_list_result = word_in_wordgroup(not_in_sent_word_list, df_youtube_sent_result, "search_string", 7, simple=False)
##df_not_in_sent_selected_word_list_result = not_in_sent_selected_word_list_result.drop(["word"], axis=1)
#df_not_in_sent_selected_word_list_result

In [27]:
#df_not_in_sent_selected_word_list_result.to_excel("Sentence_Sample.xlsx", index=False)

In [28]:
#df_not_in_sent_selected_videoid_list = df_not_in_sent_selected_word_list_result.groupby("word")[["video_id"]].agg(lambda x: set(x))
#df_not_in_sent_selected_videoid_list.reset_index(inplace=True)
#df_not_in_sent_selected_videoid_list.sort_values(by="video_id", ascending=False, inplace=True, key=lambda x:x.str.len())
#df_not_in_sent_selected_videoid_list.reset_index(drop=True, inplace=True)
#df_not_in_sent_selected_videoid_list

In [30]:
df_not_in_sent_selected_videoid_list = df_not_in_sent_selected_word_list_result.groupby(["video_id","sentence"])[["word"]].agg(lambda x: set(x))
df_not_in_sent_selected_videoid_list.reset_index(inplace=True)
df_not_in_sent_selected_videoid_list.sort_values(by="word", ascending=False, inplace=True, key=lambda x:x.str.len())
df_not_in_sent_selected_videoid_list.reset_index(drop=True, inplace=True)
df_not_in_sent_selected_videoid_list

,video_id,sentence,word
0,SjVWia8J9JI,demiştim sana iyi olacak diye bak demiştim,"{bak, olacak, iyi, sana}"
1,jrpuIZdDapQ,sizin iş de olacak gibi bak ben sana deyivereyim,"{iş, bak, olacak, sana}"
2,v9qRUChLss0,ben bak hiç iyi değilim hiç iyi değilim ben,"{bak, değilim, iyi}"
3,4UhIXjsL79Y,olacak iş değil kız da çok arsızmış,"{iş, kız, olacak}"
4,Aw5ynnoQnQE,bize saat lazım değil bak bizim saatimiz var g...,"{bak, bize, lazım}"
...,...,...,...
35441,LloriBT7484,i şte şu tarafa da bak büyük bir fırında yavaş...,{bak}
35442,Lll8fnbV4Ew,ay bu kız bu kız,{kız}
35443,LlGqbWl6Rm0,büyük ihtimalle yaptığım en iyi ve en eğlencel...,{iyi}
35444,LkiG4KFHrvY,en iyi korunma yöntemi en iyi ve tek korunma y...,{iyi}


In [39]:
videoid_list_group_list = df_not_in_sent_selected_videoid_list["video_id"].to_list()
videoid_list_group_list

[{'05PK5asBrP0',
  'kCf15d2KbBo',
  'qI3ApuLihD0',
  'RPJg_lUfHvI',
  '2pV7DNjAHKY',
  'yBVQR9eUXxQ',
  '8iUkBU1vsDw',
  'r8kx0HsfasU',
  'xF0qs4EYrM8',
  'kSIzlvs8ngw',
  'C2xtDX5qexc',
  'EMlNstwTHfU',
  'iChdwTrZeCo',
  'FZTjUj2wgsI',
  'Fr9GF0p8l_o',
  'JjwfzX7OHyc',
  '1LMDmZNFyv8',
  'GEqMgq5Hgk8',
  '3cAaBZ_Aml8',
  'd_v9LWpyiWA',
  'UxPar4k5FTY',
  '6p74_Ar_qmc',
  'nSRhszFec4Q',
  'Qx7sSetcIIQ',
  '0jB0w2DUB5s',
  'eYZA9hehwR0',
  '37A7A4ykrD4',
  '1FURmgQJYHY',
  'dSrXG27FnBY',
  '7L4X_ZuPu1Y',
  '3aUTbWogu4E',
  'VMjo4rXNgNc',
  'pB4QI5qwCN8',
  'gHjUEWlfmOw',
  'xuHjKocZTik',
  'dhnZ8qxmP00',
  'xDa_yQScau8',
  'kOwaqR6St2I',
  'QMUCHwM0CR4',
  'B115Vlbnh4w',
  'Zcrf10Vfeis',
  '1pWuXjJ1SUE',
  'C6zxXiBx5ng',
  'rEJBsxrxqec',
  'H6jzjlyJeRY',
  'G9sYOW9SU2k',
  'lDkLtdDT3ks',
  'enCsO_xQ3ZA',
  'YUcrsJAR3Ck',
  'uLM3hC_A9B0',
  'xDyuUiBsabQ',
  'dL8I12VZUHM',
  'R5Em8Z2Pfjo',
  'obhAvqN99_Y',
  'P4FZ1J_T3dI',
  '9bJSoTvdZeo',
  'SJhDWSW6Xxo',
  'nMSrzqjAFfs',
  '0IQUDH43SbU

In [40]:
set.intersection(*videoid_list_group_list)

set()

In [41]:
set_list = []
for i in range(len(df_not_in_sent_selected_videoid_list)-2):
    var_1 = videoid_list_group_list[i]
    set_list.append(var_1)
set_intersect = set.intersection(*set_list)
list_intersect = list(set_intersect) 
list_intersect   

[]

In [104]:
df_intersect_result = df_not_in_sent_selected_word_list_result[df_not_in_sent_selected_word_list_result["video_id"].isin(list_intersect)]
df_intersect_result

,word,search_string,start_time,end_time,sentence,video_id,video_url


In [105]:
df_intersect_result["video_url"] = "https://www.youtube.com/watch?v="+df_intersect_result['video_id'].map(str)+"&t="+df_intersect_result['start_time'].map(str)+"s"
df_intersect_result

,word,search_string,start_time,end_time,sentence,video_id,video_url


In [106]:
# Option
#df_intersect_sent_count = df_intersect_result.groupby("search_string")[["search_string"]].count()
df_intersect_sent_count = df_intersect_result.groupby(["search_string","start_time","end_time","sentence","video_id","video_url"])[["search_string"]].count()
df_intersect_sent_count.rename(columns={"search_string":"count"}, inplace=True)
df_intersect_sent_count.reset_index(inplace=True)
df_intersect_sent_count.sort_values(by="count", ascending=False, inplace=True)
df_intersect_sent_count

,search_string,start_time,end_time,sentence,video_id,video_url,count


In [107]:
df_intersect_sent_word_join = df_intersect_result.groupby(["search_string","start_time","end_time","sentence","video_id","video_url"])["word"].apply(lambda x: ','.join(x)).reset_index()
df_intersect_sent_word_join.drop_duplicates(inplace=True)
df_intersect_sent_word_join.sort_values(by="word", ascending=False, inplace=True, key=lambda x:x.str.len())
df_intersect_sent_word_join

,search_string,start_time,end_time,sentence,video_id,video_url,word


In [70]:
df_intersect_sent_word_join.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Group_In_Youtube_Sentence_Word_Index_List_Sample4.xlsx", index=False)

#### Remain Words

In [31]:
remain_word_list = ["merhaba","pekala"]

In [38]:
df_not_in_sent_selected_videoid_list_remain = df_not_in_sent_selected_videoid_list[df_not_in_sent_selected_videoid_list["word"].isin(remain_word_list)]
df_not_in_sent_selected_videoid_list_remain

,word,video_id
6,merhaba,"{bTDD2nj0BYM, LQ7ispbi_6U, 9GbYnL0tvcg, 15AOCt..."
7,pekala,"{xcitAtuxPCk, d9tZZGbLQMU, hLOOwsxDBJo, qmMZmy..."


In [39]:
videoid_list_group_list_remain = df_not_in_sent_selected_videoid_list_remain["video_id"].to_list()
videoid_list_group_list_remain

[{'0JqaxCjCQKQ',
  '15AOCtzI1cM',
  '1OzBRPolzfU',
  '1iByM08MvqU',
  '2SgmBwBSt8M',
  '480nF81vFI0',
  '4lJ0vD6QO1o',
  '5EsyiXMKOPs',
  '5g7hQG2MQNw',
  '6XgenY_eEro',
  '6_o14WcPUC8',
  '6eOnUHKbZ2g',
  '7H4jvc3ERrc',
  '7gw4cgfRxPM',
  '8OjNg0Sspro',
  '8dvpd87QV4s',
  '94KUKlC1rlc',
  '9GbYnL0tvcg',
  '9JVCjzcvXco',
  'ALQa8SjBujc',
  'AvfpnsqaWw4',
  'AxEi0Uy8gMQ',
  'B8TM4eoaSk8',
  'Bbaal2Neohk',
  'DPMTVjWy1E4',
  'F2iRjNS9KCk',
  'Fc4drtbgDb0',
  'G1WBHh_uWF4',
  'GWsDgpOaPH8',
  'HBxB7ytsEus',
  'HN02LoRFSy8',
  'HWxPjFUR2Qg',
  'HXB7HUg7Mks',
  'IF78KAsksbQ',
  'J1sEju6i5iQ',
  'JWrpgQhFgK8',
  'JrcKPsflI5k',
  'K0TANHoBncY',
  'KVCZbb3svSc',
  'Ks19MD2LRUw',
  'Kx2GhRVfgyg',
  'L9bkiAADDIY',
  'LJiP_ouvA9k',
  'LQ7ispbi_6U',
  'M3ELr8oCr40',
  'MWMmHcPKvfs',
  'MgIFbO_BmaU',
  'NbdXFQECU14',
  'NeMTKw_SZRc',
  'O3iT4_8jUVw',
  'OALASDJOKb8',
  'OPWhkKQLTmE',
  'Oz9wJkWAEec',
  'P_KbNZi5uUo',
  'RUscR5_LL88',
  'Ri8OHAHmTJw',
  'Rm6zwwnAVeA',
  'SA3R1HniCoM',
  'Sscep6rpD_E

In [40]:
set_intersect2 = set.intersection(*videoid_list_group_list_remain)
set_intersect2

set()

In [33]:
df_not_in_sent_selected_word_remain_result = word_in_wordgroup(["merhaba","pekala"], df_youtube_sent_result, "search_string", 10, simple=False)
#df_not_in_sent_selected_word_remain_result = not_in_sent_selected_word_list_result.drop(["word"], axis=1)
df_not_in_sent_selected_word_remain_result

,word,search_string,start_time,end_time,sentence,video_id,video_url
0,merhaba,merhaba merhaba merhaba merhaba merhaba uzun ve,517,522,dostlar merhaba merhaba merhaba merhaba merhab...,VRPrnvtZjvg,https://www.youtube.com/watch?v=VRPrnvtZjvg&t=...
1,merhaba,çok güzel burada sen burada merhaba sen bizim,1887,1891,uğraşıyor survivorda merhaba serman seni görme...,9GbYnL0tvcg,https://www.youtube.com/watch?v=9GbYnL0tvcg&t=...
2,merhaba,gel gel hadi gel hadi teşekkürler merhaba,6758,6760,gel gel hadi gel hadi teşekkürler merhaba jale,bt7g4VoVBoE,https://www.youtube.com/watch?v=bt7g4VoVBoE&t=...
3,merhaba,değilim bu peki onları burada merhaba onu,2151,2155,gidişat hakkında şu an emin değilim bu peki on...,1iByM08MvqU,https://www.youtube.com/watch?v=1iByM08MvqU&t=...
4,merhaba,var burada burada göre olabilir merhaba,99,102,hoparlörleri deyince sınıfından kaan var burad...,JrcKPsflI5k,https://www.youtube.com/watch?v=JrcKPsflI5k&t=99s
5,merhaba,efendim ben de merhaba tabii ki efendim,4230,4233,bakabilir miyim efendim ben de merhaba tabii k...,8OjNg0Sspro,https://www.youtube.com/watch?v=8OjNg0Sspro&t=...
6,merhaba,merhaba çok teşekkür ederim bu benim,2249,2253,hepsini değiştirir sen yemezsin ali merhaba ço...,nPVd4XaeM34,https://www.youtube.com/watch?v=nPVd4XaeM34&t=...
7,merhaba,merhaba çok teşekkür ederim buraya,3275,3278,merhaba tolga merhaba çok teşekkür ederim bura...,AvfpnsqaWw4,https://www.youtube.com/watch?v=AvfpnsqaWw4&t=...
8,merhaba,sadece merhaba merhaba o kadar ama,5574,5577,arkadaşım değil büyük hanım aynı bölükteydik s...,VBdcD7hLGuo,https://www.youtube.com/watch?v=VBdcD7hLGuo&t=...
9,merhaba,biz sadece bir merhaba demek için,5612,5614,biz sadece bir merhaba demek için geldik,MWMmHcPKvfs,https://www.youtube.com/watch?v=MWMmHcPKvfs&t=...


In [34]:
df_not_in_sent_selected_word_remain_result.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Group_In_Youtube_Sentence_Word_Index_List_Sample5.xlsx", index=False)

#### Copy Move And Delete

In [41]:
output_file = glob.glob(f"{lang_folder.capitalize()}_{word_end}_Word_Group_In_Youtube_Sentence_*_Sample*.xlsx")
output_file

['Turkish_200_Word_Group_In_Youtube_Sentence_Word_Index_List_Sample4.xlsx',
 'Turkish_200_Word_Group_In_Youtube_Sentence_Word_Index_List_Sample5.xlsx',
 'Turkish_200_Word_Group_In_Youtube_Sentence_Word_Index_List_Sample_Selected.xlsx']

In [42]:
for k in output_file:
    source = k # source directory
    destination = path
    shutil.copy2(source, destination)

In [43]:
for i in output_file:
    try:
        os.remove(i)
    except:
        pass

#### Deploy File Word Count

In [44]:
df_deploy_file = pd.read_excel(f"{source_path}/Turkish_200_Word_Group_In_69_Youtube_Sentence_Deploy.xlsx")
df_deploy_file

,search_string,start_time,end_time,sentence,video_id,video_url,word_index_list
0,kendi şimdi sen mi ne oluyor biliyor musun bu ...,446,451,artık hamill olması lazım altı kendi şimdi sen...,Yuu5VhJBIUg,https://www.youtube.com/watch?v=Yuu5VhJBIUg&t=...,"[133, 43, 19, 5, 2, 139, 109, 53, 1, 43, 168, ..."
1,gerçekten uzun ve bu da benim için çok önemli ...,623,627,bu tabletin şarj ömrü batarya ömrü gerçekten u...,r_9ngjGh2Ks,https://www.youtube.com/watch?v=r_9ngjGh2Ks&t=...,"[70, 172, 3, 1, 15, 31, 4, 9, 156, 68, 67]"
2,her zaman öyle değil herkes için de her zaman ...,559,562,her zaman öyle değil herkes için de her zaman ...,cPoQ27hCNOU,https://www.youtube.com/watch?v=cPoQ27hCNOU&t=...,"[30, 35, 46, 14, 173, 4, 8, 30, 35, 46, 14]"
3,bile yok bu da benim için aslında önemli bir ş...,131,134,bulabilirsiniz şu an görünce bile yok bu da be...,aqQ7IUho9pA,https://www.youtube.com/watch?v=aqQ7IUho9pA&t=...,"[59, 22, 1, 15, 31, 4, 174, 156, 0, 16, 68]"
4,için artık değil misin gerek yok bunun için si...,65,70,göndermek için artık değil misin gerek yok bun...,gX77n13Qj0o,https://www.youtube.com/watch?v=gX77n13Qj0o&t=65s,"[4, 61, 14, 49, 117, 22, 97, 4, 121, 0]"
...,...,...,...,...,...,...,...
64,gerçek mi ya ne,1471,1473,gerçek mi ya ne,kulZ_sWzBR0,https://www.youtube.com/watch?v=kulZ_sWzBR0&t=...,gerçek
65,bence olur bence olmaz,3893,3895,bence olur bence olmaz,MhHCQ_cb3T8,https://www.youtube.com/watch?v=MhHCQ_cb3T8&t=...,bence
66,ben asla öyle bir şey,4768,4769,ben asla öyle bir şey yapmadım,kulZ_sWzBR0,https://www.youtube.com/watch?v=kulZ_sWzBR0&t=...,asla
67,bir şey yok ki ya adam,2554,2556,bir şey yok ki ya adam hırsızın teki çıktı işte,MhHCQ_cb3T8,https://www.youtube.com/watch?v=MhHCQ_cb3T8&t=...,adam


In [46]:
df_deploy_file_count = word_count_result(df_deploy_file, df_deploy_file.columns[0:1])
df_deploy_file_count.sort_values(by="word_count", ascending=False, inplace=True)
df_deploy_file_count

,word,word_count
0,bir,15
1,çok,11
2,tamam,8
3,değil,7
4,şey,7
...,...,...
130,sizi,1
129,seni,1
128,yeni,1
127,selam,1
